In [1]:
import warnings
warnings.filterwarnings('ignore')

In [35]:
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
import joblib

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [5]:
#Se renombra la base de Training a Df para el desarrollo del proyect
df = dataTraining

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7895 entries, 3107 to 215
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    7895 non-null   int64  
 1   title   7895 non-null   object 
 2   plot    7895 non-null   object 
 3   genres  7895 non-null   object 
 4   rating  7895 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 370.1+ KB


In [7]:
# busqueda valorres perdidos o NA
df.isna().any()

year      False
title     False
plot      False
genres    False
rating    False
dtype: bool

In [8]:
df['genres'].value_counts()

['Drama']                                                            429
['Comedy']                                                           368
['Comedy', 'Drama', 'Romance']                                       306
['Comedy', 'Romance']                                                291
['Comedy', 'Drama']                                                  287
                                                                    ... 
['Biography', 'Musical', 'Romance', 'War']                             1
['Comedy', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Thriller']      1
['Crime', 'Drama', 'Fantasy', 'Mystery', 'Thriller']                   1
['Documentary', 'History', 'News']                                     1
['Mystery', 'Horror', 'Sci-Fi']                                        1
Name: genres, Length: 1336, dtype: int64

In [9]:
df['rating'].describe()

count    7895.000000
mean        6.402812
std         1.078260
min         1.200000
25%         5.800000
50%         6.500000
75%         7.200000
max         9.300000
Name: rating, dtype: float64

In [10]:
df.shape[0]

7895

In [11]:
df = df.drop_duplicates(subset=['plot'])

In [12]:
df.shape[0]

7894

In [13]:
df['genres'] = df['genres'].map(lambda x: eval(x))
mlb = MultiLabelBinarizer()
genres = df['genres']
labels = mlb.fit_transform(genres)
label_classes = mlb.classes_

In [14]:
label_classes

array(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
       'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War',
       'Western'], dtype=object)

In [15]:
#Preprocesamiento
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\afval\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\afval\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
import spacy, re
nlp = spacy.load('es_core_news_sm')
wordnet_lemmatizer = WordNetLemmatizer()

def clean_plot(plot):      
    letters_only = re.sub("[^a-zA-Z]", " ", plot) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))   
    meaningful_words = [wordnet_lemmatizer.lemmatize(w) for w in words if not w in stops]   
    return( " ".join( meaningful_words ))

df['plot'] = df['plot'].apply(clean_plot)

In [17]:
def lemmatize_text(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return ' '.join(lemmas)

df['plot'] = df['plot'].apply(lemmatize_text)

In [18]:
# Crear la matriz de características TF-IDF
vectorizer = TfidfVectorizer(max_features = 3250, stop_words ='english', smooth_idf=True, use_idf= True, sublinear_tf=True, norm='l1', analyzer='word', strip_accents='unicode')
X = vectorizer.fit_transform(df['plot'])
#vectorizer = TfidfVectorizer(max_features = 3250, stop_words ='english', smooth_idf=True, use_idf= True, sublinear_tf=True, norm='l1', analyzer='word')

In [19]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X, labels, test_size=0.33, random_state=42)

In [20]:
from sklearn.linear_model import LogisticRegression

# Entrenar y evaluar el clasificador de regresión logística multilabel
lr_classifier = OneVsRestClassifier(LogisticRegression())
lr_classifier.fit(X_train, y_train_genres)
lr_predictions = lr_classifier.predict_proba(X_test)
lr_roc_auc = roc_auc_score(y_test_genres, lr_predictions, average='macro')
print('ROC-AUC para el clasificador de regresión logística multilabel:', lr_roc_auc)

ROC-AUC para el clasificador de regresión logística multilabel: 0.8660323361760129


In [22]:
# Entrenar y evaluar el clasificador de bosques aleatorios multilabel
rf_classifier = OneVsRestClassifier(RandomForestClassifier())
rf_classifier.fit(X_train, y_train_genres)
rf_predictions = rf_classifier.predict_proba(X_test)
rf_roc_auc = roc_auc_score(y_test_genres, rf_predictions, average='macro')
print('ROC-AUC para el clasificador de bosques aleatorios multilabel:', rf_roc_auc)

ROC-AUC para el clasificador de bosques aleatorios multilabel: 0.7991153388443131


In [24]:
from sklearn.svm import SVC

# Entrenar y evaluar el clasificador de vectores de soporte multilabel
svm_classifier = OneVsRestClassifier(SVC(probability=True))
svm_classifier.fit(X_train, y_train_genres)
svm_predictions = svm_classifier.predict_proba(X_test)
svm_roc_auc = roc_auc_score(y_test_genres, svm_predictions, average='macro')
print('ROC-AUC para el clasificador de vectores de soporte multilabel:', svm_roc_auc)

ROC-AUC para el clasificador de vectores de soporte multilabel: 0.8550922480263158


In [46]:
from sklearn.ensemble import GradientBoostingClassifier

# Entrenar y evaluar el clasificador AdaBoost multilabel
gradient_classifier = OneVsRestClassifier(GradientBoostingClassifier())
gradient_classifier.fit(X_train, y_train_genres)
gradient_predictions = gradient_classifier.predict_proba(X_test)
gradient_roc_auc = roc_auc_score(y_test_genres, gradient_predictions, average='macro')
print('ROC-AUC para el clasificador AdaBoost multilabel:', gradient_roc_auc)

ROC-AUC para el clasificador AdaBoost multilabel: 0.775806704599192


In [29]:
from xgboost import XGBClassifier

# Entrenar y evaluar el clasificador XGBoost multilabel
xgboost_classifier = OneVsRestClassifier(XGBClassifier())
xgboost_classifier.fit(X_train, y_train_genres)
xgboost_predictions = xgboost_classifier.predict_proba(X_test)
xgboost_roc_auc = roc_auc_score(y_test_genres, xgboost_predictions, average='macro')
print('ROC-AUC para el clasificador XGBoost multilabel:', xgboost_roc_auc)

ROC-AUC para el clasificador XGBoost multilabel: 0.8177264060644891


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier


In [25]:
# Clasificador Logistic Regression
logreg_clf = OneVsRestClassifier(LogisticRegression())
logreg_params = {
    'estimator__C': [0.01, 1, 10],
}
logreg_grid_search = GridSearchCV(logreg_clf, param_grid=logreg_params, scoring='roc_auc', cv=3)
logreg_grid_search.fit(X_train, y_train_genres)
logreg_best_params = logreg_grid_search.best_params_
logreg_best_score = logreg_grid_search.best_score_
logreg_best_classifier = logreg_clf.set_params(**logreg_best_params)
logreg_best_classifier.fit(X_train, y_train_genres)
logreg_predictions = logreg_best_classifier.predict_proba(X_test)
logreg_roc_auc = roc_auc_score(y_test_genres, logreg_predictions, average='macro')
print("Mejores parámetros para Logistic Regression:")
print(logreg_best_params)
print("Mejor ROC-AUC obtenido:", logreg_best_score)
print("ROC-AUC en el conjunto de prueba:", logreg_roc_auc)
print("")

Mejores parámetros para Logistic Regression:
{'estimator__C': 10}
Mejor ROC-AUC obtenido: 0.8639757336443686
ROC-AUC en el conjunto de prueba: 0.8696794512551712



In [25]:
# Clasificador Random Forest
rf_clf = OneVsRestClassifier(RandomForestClassifier())
rf_params = {'estimator__n_estimators': [50, 100, 200], 'estimator__max_depth': [None, 5, 10]}
rf_grid_search = GridSearchCV(rf_clf, param_grid=rf_params, scoring='roc_auc', cv=3)
rf_grid_search.fit(X_train, y_train_genres)
rf_best_params = rf_grid_search.best_params_
rf_best_score = rf_grid_search.best_score_
rf_best_classifier = rf_clf.set_params(**rf_best_params)
rf_best_classifier.fit(X_train, y_train_genres)
rf_predictions = rf_best_classifier.predict_proba(X_test)
rf_roc_auc = roc_auc_score(y_test_genres, rf_predictions)
print("Mejores parámetros para Random Forest:")
print(rf_best_params)
print("Mejor ROC-AUC obtenido:", rf_best_score)
print("ROC-AUC en el conjunto de prueba:", rf_roc_auc)
print("")

Mejores parámetros para Random Forest:
{'estimator__max_depth': 5, 'estimator__n_estimators': 200}
Mejor ROC-AUC obtenido: 0.8164077219736993
ROC-AUC en el conjunto de prueba: 0.8166600173538866



In [26]:
# Clasificador SVM
svm_clf = OneVsRestClassifier(SVC(probability=True))
svm_params = {'estimator__C': [0.1, 1, 10], 'estimator__kernel': ['linear', 'rbf']}
svm_grid_search = GridSearchCV(svm_clf, param_grid=svm_params, scoring='roc_auc', cv=3)
svm_grid_search.fit(X_train, y_train_genres)
svm_best_params = svm_grid_search.best_params_
svm_best_score = svm_grid_search.best_score_
svm_best_classifier = svm_clf.set_params(**svm_best_params)
svm_best_classifier.fit(X_train, y_train_genres)
svm_predictions = svm_best_classifier.predict_proba(X_test)
svm_roc_auc = roc_auc_score(y_test_genres, svm_predictions)
print("Mejores parámetros para SVM:")
print(svm_best_params)
print("Mejor ROC-AUC obtenido:", svm_best_score)
print("ROC-AUC en el conjunto de prueba:", svm_roc_auc)
print("")

Mejores parámetros para SVM:
{'estimator__C': 1, 'estimator__kernel': 'rbf'}
Mejor ROC-AUC obtenido: 0.8561128827702221
ROC-AUC en el conjunto de prueba: 0.8550924512257064



In [27]:
# Clasificador AdaBoost
adaboost_clf = OneVsRestClassifier(AdaBoostClassifier())
adaboost_params = {'estimator__n_estimators': [50, 100, 200]}
adaboost_grid_search = GridSearchCV(adaboost_clf, param_grid=adaboost_params, scoring='roc_auc', cv=3)
adaboost_grid_search.fit(X_train, y_train_genres)
adaboost_best_params = adaboost_grid_search.best_params_
adaboost_best_score = adaboost_grid_search.best_score_
adaboost_best_classifier = adaboost_clf.set_params(**adaboost_best_params)
adaboost_best_classifier.fit(X_train, y_train_genres)
adaboost_predictions = adaboost_best_classifier.predict_proba(X_test)
adaboost_roc_auc = roc_auc_score(y_test_genres, adaboost_predictions)
print("Mejores parámetros para AdaBoost:")
print(adaboost_best_params)
print("Mejor ROC-AUC obtenido:", adaboost_best_score)
print("ROC-AUC en el conjunto de prueba:", adaboost_roc_auc)
print("")

Mejores parámetros para AdaBoost:
{'estimator__n_estimators': 100}
Mejor ROC-AUC obtenido: 0.754864961491434
ROC-AUC en el conjunto de prueba: 0.7706821086925469



In [28]:
# Clasificador XGBoost
xgboost_clf = OneVsRestClassifier(XGBClassifier())
xgboost_params = {'estimator__n_estimators': [50, 100, 200], 'estimator__max_depth': [3, 5, 7]}
xgboost_grid_search = GridSearchCV(xgboost_clf, param_grid=xgboost_params, scoring='roc_auc', cv=3)
xgboost_grid_search.fit(X_train, y_train_genres)
xgboost_best_params = xgboost_grid_search.best_params_
xgboost_best_score = xgboost_grid_search.best_score_
xgboost_best_classifier = xgboost_clf.set_params(**xgboost_best_params)
xgboost_best_classifier.fit(X_train, y_train_genres)
xgboost_predictions = xgboost_best_classifier.predict_proba(X_test)
xgboost_roc_auc = roc_auc_score(y_test_genres, xgboost_predictions)
print("Mejores parámetros para XGBoost:")
print(xgboost_best_params)
print("Mejor ROC-AUC obtenido:", xgboost_best_score)
print("ROC-AUC en el conjunto de prueba:", xgboost_roc_auc)
print("")

Mejores parámetros para XGBoost:
{'estimator__max_depth': 3, 'estimator__n_estimators': 100}
Mejor ROC-AUC obtenido: 0.8076177608993529
ROC-AUC en el conjunto de prueba: 0.8224578374704703



### Evaluación de modelos

### Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression

# Entrenar y evaluar el clasificador de regresión logística multilabel
lr_classifier = OneVsRestClassifier(LogisticRegression(C=10, random_state=42))
lr_classifier.fit(X_train, y_train_genres)
lr_predictions = lr_classifier.predict_proba(X_test)
lr_roc_auc = roc_auc_score(y_test_genres, lr_predictions, average='macro')
print('ROC-AUC para el clasificador de regresión logística multilabel:', lr_roc_auc)

ROC-AUC para el clasificador de regresión logística multilabel: 0.8696794512551712


In [23]:
lr_predictions_train = lr_classifier.predict_proba(X_train)
lr_predictions_test = lr_classifier.predict_proba(X_test)
lr_roc_auc_train = roc_auc_score(y_train_genres, lr_predictions_train, average='macro')
lr_roc_auc_test = roc_auc_score(y_test_genres, lr_predictions_test, average='macro')
print('ROC-AUC train:', lr_roc_auc_train)
print('ROC-AUC test:', lr_roc_auc_test)

ROC-AUC train: 0.9612897726848605
ROC-AUC test: 0.8696794512551712


#### Ramdon Forest

In [27]:
# Clasificador Random Forest
# Entrenar y evaluar el clasificador de bosques aleatorios multilabel
rf_classifier = OneVsRestClassifier(RandomForestClassifier(max_depth=5, n_estimators=200))
rf_classifier.fit(X_train, y_train_genres)
rf_predictions = rf_classifier.predict_proba(X_test)
rf_roc_auc = roc_auc_score(y_test_genres, rf_predictions, average='macro')
print('ROC-AUC para el clasificador de bosques aleatorios multilabel:', rf_roc_auc)

ROC-AUC para el clasificador de bosques aleatorios multilabel: 0.8165825261410457


In [29]:
rf_predictions_train = rf_classifier.predict_proba(X_train)
rf_predictions_test = rf_classifier.predict_proba(X_test)
rf_roc_auc_train = roc_auc_score(y_train_genres, rf_predictions_train, average='macro')
rf_roc_auc_test = roc_auc_score(y_test_genres, rf_predictions_test, average='macro')
print('ROC-AUC train:', rf_roc_auc_train)
print('ROC-AUC test:', rf_roc_auc_test)

ROC-AUC train: 0.9517430411286929
ROC-AUC test: 0.8165825261410457


### SVC

In [30]:
from sklearn.svm import SVC

# Entrenar y evaluar el clasificador de vectores de soporte multilabel
svm_classifier = OneVsRestClassifier(SVC(C=1, kernel= 'rbf',probability=True))
svm_classifier.fit(X_train, y_train_genres)
svm_predictions = svm_classifier.predict_proba(X_test)
svm_roc_auc = roc_auc_score(y_test_genres, svm_predictions, average='macro')
print('ROC-AUC para el clasificador de vectores de soporte multilabel:', svm_roc_auc)

ROC-AUC para el clasificador de vectores de soporte multilabel: 0.8550848907399508


In [31]:
svm_predictions_train = svm_classifier.predict_proba(X_train)
svm_predictions_test = svm_classifier.predict_proba(X_test)
svm_roc_auc_train = roc_auc_score(y_train_genres, svm_predictions_train, average='macro')
svm_roc_auc_test = roc_auc_score(y_test_genres, svm_predictions_test, average='macro')
print('ROC-AUC train:', svm_roc_auc_train)
print('ROC-AUC test:', svm_roc_auc_test)

ROC-AUC train: 0.9986992745897076
ROC-AUC test: 0.8550848907399508


### XGBoost

In [32]:
%%time
from xgboost import XGBClassifier

# Entrenar y evaluar el clasificador XGBoost multilabel
xgboost_classifier = OneVsRestClassifier(XGBClassifier(max_depth=3, n_estimators=100))
xgboost_classifier.fit(X_train, y_train_genres)
xgboost_predictions = xgboost_classifier.predict_proba(X_test)
xgboost_roc_auc = roc_auc_score(y_test_genres, xgboost_predictions, average='macro')
print('ROC-AUC para el clasificador XGBoost multilabel:', xgboost_roc_auc)

ROC-AUC para el clasificador XGBoost multilabel: 0.8224578374704703
Wall time: 15.3 s


In [33]:
xgboost_predictions_train = xgboost_classifier.predict_proba(X_train)
xgboost_predictions_test = xgboost_classifier.predict_proba(X_test)
xgboost_roc_auc_train = roc_auc_score(y_train_genres, xgboost_predictions_train, average='macro')
xgboost_roc_auc_test = roc_auc_score(y_test_genres, xgboost_predictions_test, average='macro')
print('ROC-AUC train:', xgboost_roc_auc_train)
print('ROC-AUC test:', xgboost_roc_auc_test)

ROC-AUC train: 0.9794366281057633
ROC-AUC test: 0.8224578374704703


In [38]:
## guardar modelos
joblib.dump(lr_classifier, 'C:/Users/afval/Documents/MIAD_ML/MIAD_ML_NLP_2023/Semana 7/genres_movie_lr.pkl', compress=3)
joblib.dump(rf_classifier, 'C:/Users/afval/Documents/MIAD_ML/MIAD_ML_NLP_2023/Semana 7/genres_movie_rf.pkl', compress=3)
joblib.dump(svm_classifier, 'C:/Users/afval/Documents/MIAD_ML/MIAD_ML_NLP_2023/Semana 7/genres_movie_svm.pkl', compress=3)
joblib.dump(xgboost_classifier, 'C:/Users/afval/Documents/MIAD_ML/MIAD_ML_NLP_2023/Semana 7/genres_movie_xg.pkl', compress=3)

['C:/Users/afval/Documents/MIAD_ML/MIAD_ML_NLP_2023/Semana 7/genres_movie_xg.pkl']

In [30]:
# transformación variables predictoras X del conjunto de test
X_test_dtm = vectorizer.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = logreg_best_classifier.predict_proba(X_test_dtm)

In [31]:
# Guardar predicciones en formato exigido en la competencia de kaggle
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv(r'C:\Users\afval\Documents\MIAD_ML\MIAD_ML_NLP_2023\Semana 7\pred_genres_text_RF_7.csv', index_label='ID')
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.105521,0.089913,0.021407,0.042772,0.406041,0.120591,0.036523,0.592757,0.042820,0.101477,...,0.033528,0.095175,0.000738,0.526914,0.062622,0.012580,0.024182,0.190526,0.032263,0.030123
4,0.078858,0.034149,0.025386,0.157484,0.524592,0.215450,0.045036,0.712470,0.035041,0.034528,...,0.034827,0.052075,0.000807,0.150027,0.030283,0.012972,0.032177,0.161782,0.040822,0.028775
5,0.136352,0.075544,0.023519,0.050704,0.235746,0.366044,0.043195,0.646299,0.048997,0.060168,...,0.026663,0.174633,0.000754,0.179948,0.075507,0.011297,0.029001,0.384604,0.038018,0.026607
6,0.143232,0.102351,0.025354,0.046708,0.285588,0.120477,0.040571,0.590403,0.063092,0.067713,...,0.029495,0.108617,0.000757,0.229193,0.097250,0.010247,0.035297,0.282391,0.053962,0.026641
7,0.125484,0.097806,0.026487,0.037207,0.292163,0.151224,0.045495,0.384984,0.058389,0.104774,...,0.027186,0.118966,0.000745,0.172704,0.257945,0.011630,0.022456,0.295695,0.035522,0.024435


##### Prueba API

In [40]:
# Importar modelo y función de predicción
from model_deployment_genres.mp2_model_deployment import predict_genres

ModuleNotFoundError: No module named 'model_deployment_genres'

In [ ]:
Importación librerías
from flask import Flask
#from flask_restplus import Api, Resource, fields
from flask_restx import Api, Resource, fields

In [ ]:
app = Flask(__name__)

api = Api(
    app, 
    version='1.0', 
    title='Predicción generos de peliculas',
    description='Predicción generos de peliculas')

ns = api.namespace('predict', 
     description='Genres Classifier')
   
parser = api.parser()

parser.add_argument(
    'year',
    type = int,
    required = True)

parser.add_argument(
    'title',
    type = str,
    required = True)

parser.add_argument(
    'plot',
    type = str,
    required = True
)

parser.add_argument(
    'rating',
    type = float,
    required = True
)

resource_fields = api.model('Resource', {
    'result': fields.String,
})

In [ ]:
@ns.route('/')
class GenresApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        
        return {
         "result": predict_genre(args['year'], args['title'], args['plot'], args['rating'])
        }, 200

In [ ]:
# Ejecución de la aplicación que disponibiliza el modelo de manera local en el puerto 5000
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)